In [1]:
import os, os.path

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

import torch
import numpy as np
import pandas as pd
import re
import random

from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score, roc_auc_score
from transformers import AutoModel, AutoConfig, AutoTokenizer, get_linear_schedule_with_warmup
from tqdm.auto import tqdm
from torch.optim import Adam, AdamW
from torch.nn import functional as F

In [2]:
def set_seed(seed: int = 666, precision: int = 10) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.set_printoptions(precision=precision)

In [3]:
CLEANR = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')

def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, ' ', raw_html)
    cleantext = re.sub('\s+', ' ', cleantext)
    return cleantext

In [4]:
train_data = pd.read_csv('./train_dataset_train.csv', encoding='utf-8')
test_data = pd.read_csv('./test_dataset_test.csv', encoding='utf-8')

train_data = train_data[train_data['Категория'] != 12]

train_data['Текст Сообщения'] = train_data['Текст Сообщения'].apply(cleanhtml)
test_data['Текст Сообщения'] = test_data['Текст Сообщения'].apply(cleanhtml)

train_data['Текст Сообщения'] = train_data['Текст Сообщения'].apply(lambda x: x[:512])
test_data['Текст Сообщения'] = test_data['Текст Сообщения'].apply(lambda x: x[:512])

train_data['Текст Сообщения'] = train_data['Текст Сообщения'].apply(lambda x: '<s> ' + x + ' </s>')
test_data['Текст Сообщения'] = test_data['Текст Сообщения'].apply(lambda x: '<s> ' + x + ' </s>')

repl_dict = {key:i for i, (key, value) in enumerate(dict(train_data['Категория'].value_counts()).items())}
train_data['Категория'] = train_data['Категория'].map(repl_dict)

In [5]:
def create_dataloader(x_data, batch_size, sampler, y_data = ''):
    tokenized_texts = [tokenizer.tokenize(sent) for sent in x_data]
    inputs = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    padded_inputs = pad_sequences(inputs, 
                          dtype='long', 
                          padding='post', 
                          truncating='post')
    masks = [[float(i>0) for i in seq] for seq in padded_inputs]
    
    padded_inputs = torch.tensor(padded_inputs)
    masks = torch.tensor(masks)
    
    try:
        labels = np.array(y_data)
        labels = torch.tensor(labels)
        tensor_dataset = torch.utils.data.TensorDataset(padded_inputs, masks, labels)
    except:
        tensor_dataset = torch.utils.data.TensorDataset(padded_inputs, masks)
    
    if sampler == 'random':
        dataloader = torch.utils.data.DataLoader(
            tensor_dataset,
            batch_size=batch_size,
            sampler=torch.utils.data.RandomSampler(tensor_dataset)
        )
    elif sampler == 'sequential':
        dataloader = torch.utils.data.DataLoader(
            tensor_dataset,
            batch_size=batch_size,
            sampler=torch.utils.data.SequentialSampler(tensor_dataset)
        )
    return dataloader

In [6]:
model_name = 'DeepPavlov/xlm-roberta-large-en-ru'

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:

class myBertModel(torch.nn.Module):
    def __init__(self, hidden_neurons, n_labels):
        super(myBertModel, self).__init__()
        self.n_labels = n_labels
        config = AutoConfig.from_pretrained(model_name, num_labels=self.n_labels)
        self.model = AutoModel.from_pretrained(model_name, config=config)
        self.hidden_neurons = hidden_neurons
        self.linear1 = torch.nn.Linear(config.hidden_size, int(self.hidden_neurons / 2))
        self.classifier = torch.nn.Linear(config.hidden_size, self.n_labels)
#         self.dropout = torch.nn.Dropout(.3)
        
        self.attention = torch.nn.Sequential(
            torch.nn.Linear(config.hidden_size, 512),
            torch.nn.Tanh(),
            torch.nn.Linear(512, 1),
            torch.nn.Softmax(dim=1)
        )
        
    def forward(self, input_ids, attention_mask):
        bert_output = self.model(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_states = bert_output[0]
        weights = self.attention(last_hidden_states)
        feature = torch.sum(weights * last_hidden_states, dim=1)
        scores = self.classifier(feature)
        return scores

In [9]:
def train_loop(train_dataloader, val_dataloader, model, n_epochs, fold):
    optimizer = AdamW(model.parameters(), lr=1e-5)
    num_train_steps = int(len(list(train_dataloader)) * n_epochs)
    
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=60, num_training_steps=num_train_steps
    )
    
    prev_val_roc_auc = 0
    lrs = []
    
    for epoch in range(n_epochs):
        
        model.train()
        epoch_loss_set = []
        
        optimizer.zero_grad()

        for j, batch in enumerate(tqdm(train_dataloader)):
            
            batch = tuple(t.to(device) for t in batch)
            b_input_ids, b_input_mask, b_labels = batch
                        
            pred_probs = model.forward(b_input_ids, b_input_mask)
            
            my_loss = loss(pred_probs, b_labels)
            epoch_loss_set.append(my_loss.item())
            my_loss.backward()
                 
            pred_probs = pred_probs.detach().to('cpu').numpy()
            pred_labels = pred_probs.argmax(axis=1)
            b_labels = b_labels.to('cpu').numpy()
            
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            optimizer.zero_grad()
                 
            if j % 20 == 0 and j != 0:
                print('Epoch [{}/{}] ... Step [{}] ... LR: {:.8f} ... Grad: {:.2f} ... Mean loss: {:.4f}'.format(epoch + 1, n_epochs, j, scheduler.get_last_lr()[0], grad_norm, np.mean(epoch_loss_set)))
                print()
                
            if j == 0:
                all_train_labels = b_labels
                all_train_preds = pred_labels
            else:
                all_train_labels = np.hstack([all_train_labels, b_labels])
                all_train_preds = np.hstack([all_train_preds, pred_labels])
                                
        train_f1 = f1_score(all_train_labels, all_train_preds, average='weighted')
        
        map_labels = {real_label: i for i, real_label in enumerate(set(all_train_labels))}
        converted_all_train_labels = [map_labels[val] for val in all_train_labels]
        converted_all_train_preds = [map_labels[val] for val in all_train_preds]
            
        train_roc_auc = roc_auc_score(converted_all_train_labels, one_label_to_many(converted_all_train_preds, len(map_labels)), multi_class='ovo')
        
        print()
        print('Epoch [{}/{}] ... Train ROC-AUC: {:.2f} ... Train f1: {:.2f} ... Mean train loss: {:.4f}'.format(epoch + 1, n_epochs, train_roc_auc, train_f1, np.mean(epoch_loss_set)))
        print()
        
        all_val_preds, current_val_roc_auc = validation_loop(val_dataloader, model)
        
        if current_val_roc_auc > prev_val_roc_auc:
            prev_val_roc_auc = current_val_roc_auc
            torch.save(model.state_dict(), f'./folds_models/xlm-roberta_model_{fold}_{epoch}.pt')
            continue
        else:
            model.load_state_dict(torch.load(f'./folds_models/xlm-roberta_model_{fold}_{epoch - 1}.pt'))
            break
    return model
        
def validation_loop(val_dataloader, model):
    val_loss_set = []

    model.eval()

    for j, batch in enumerate(val_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            pred_probs = model.forward(b_input_ids, b_input_mask)
            val_loss = loss(pred_probs, b_labels)
            val_loss_set.append(val_loss.item())

            b_labels = b_labels.to('cpu').numpy()
            pred_probs = pred_probs.detach().to('cpu').numpy()
            pred_labels = pred_probs.argmax(axis=1)

        if j == 0:
            all_val_labels = b_labels
            all_val_preds = pred_labels
        else:
            all_val_labels = np.hstack([all_val_labels, b_labels])
            all_val_preds = np.hstack([all_val_preds, pred_labels])
    
    val_f1 = f1_score(all_val_labels, all_val_preds, average='weighted')
    
    map_labels = {real_label: i for i, real_label in enumerate(set(all_val_labels))}
    converted_all_val_labels = [map_labels[val] for val in all_val_labels]
    converted_all_val_preds = [map_labels[val] for val in all_val_preds]
    
    val_roc_auc = roc_auc_score(converted_all_val_labels, one_label_to_many(converted_all_val_preds, len(map_labels)), multi_class='ovo')
    print()
    print('Val ROC-AUC {:.2f} ... Val f1: {:.2f} ... Mean val loss: {:.4f}'.format(val_roc_auc, val_f1, np.mean(val_loss_set)))
    print()
    print('-------------------------------------------------------------------------------------------------------------------------')
    print()
    return all_val_preds, val_roc_auc

def do_test_preds(model, test_dataloader):
    model.eval()
    test_probs = torch.empty(0, n_labels)
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask = batch
        with torch.no_grad():
            pred_probs = model.forward(b_input_ids, b_input_mask)
            pred_probs = pred_probs.detach().to('cpu')
            test_probs = torch.cat([test_probs, pred_probs])
    return test_probs.numpy()

In [10]:
x_train, y_train = train_data['Текст Сообщения'], train_data['Категория']
x_test = test_data['Текст Сообщения']

n_labels = y_train.nunique()

loss_set = []
val_loss_set = []

n_folds = 7
n_epochs = 10
hidden_neurons = 512
batch_size = 24
max_grad_norm = 2
loss = torch.nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
val_roc_auc_treshold = 0.74
global_seed = 145
seed_step = 100

folds = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=global_seed)
splits = folds.split(x_train, y_train)
y_probs = np.zeros([x_test.shape[0], n_labels])
y_oof = np.zeros(x_train.shape[0])
f1 = 0
roc_auc = 0

test_dataloader = create_dataloader(x_test, batch_size, 'sequential')

In [11]:
def one_label_to_many(df, n_labels):
    classes = [x for x in range(n_labels)]
    y_test = []
    min_class = min(classes)
    count_class = len(classes)
    for ll in df:
        mass = [0] * count_class
        mass[int(ll) - min_class] = 1
        y_test.append(mass)

    return y_test

In [12]:
def train_val_loop(train_dataloader, val_dataloader, n_epochs, fold_n):
    model = myBertModel(hidden_neurons=hidden_neurons, n_labels=n_labels)
    model.to(device)
    model = train_loop(train_dataloader, val_dataloader, model, n_epochs, fold_n)
    val_preds, val_roc_auc = validation_loop(val_dataloader, model)
    return model, val_preds, val_roc_auc

In [13]:
for fold_n, (train_index, valid_index) in enumerate(splits):
    global_seed += fold_n * seed_step
    set_seed(global_seed)
    
    X_tr, X_val = x_train.iloc[train_index].to_numpy(), x_train.iloc[valid_index].to_numpy()
    y_tr, y_val = y_train.iloc[train_index].to_numpy(), y_train.iloc[valid_index].to_numpy()
    
    train_dataloader = create_dataloader(X_tr, batch_size, 'sequential', y_tr)
    val_dataloader = create_dataloader(X_val, batch_size, 'sequential', y_val)
    print(f"Fold [{fold_n + 1}/{n_folds}]")
    
    model, val_preds, val_roc_auc = train_val_loop(train_dataloader, val_dataloader, n_epochs, fold_n)
    y_test_probs = do_test_preds(model, test_dataloader)
    
    candidates_val_preds = [val_preds]
    candidates_test_probs = [y_test_probs]
    candidates_val_roc_aucs = np.array(val_roc_auc)
    
    n_rep = 0
    while val_roc_auc <= val_roc_auc_treshold:
        if n_rep >= 5:
            break
        print(f"Fold [{fold_n + 1}/{n_folds}] is repeated")
        model, val_preds, val_roc_auc = train_val_loop(train_dataloader, val_dataloader, n_epochs, fold_n)
        y_test_probs = do_test_preds(model, test_dataloader)
        
        candidates_val_preds.append(val_preds)
        candidates_test_probs.append(y_test_probs)
        candidates_val_roc_aucs = np.append(candidates_val_roc_aucs, val_roc_auc)
        n_rep += 1
        
    max_ind_val_roc_auc = candidates_val_roc_aucs.argmax()
    val_preds = candidates_val_preds[max_ind_val_roc_auc]
    y_test_probs = candidates_test_probs[max_ind_val_roc_auc]
    
    y_oof[valid_index] = val_preds
    f1 += f1_score(y_val, val_preds, average='weighted') / n_folds
    
    map_labels = {real_label: i for i, real_label in enumerate(set(y_val))}
    converted_y_val = [map_labels[val] for val in y_val]
    converted_val_preds = [map_labels[val] for val in val_preds]
    
    roc_auc += roc_auc_score(converted_y_val, one_label_to_many(converted_val_preds, len(map_labels)), multi_class='ovo') / n_folds
    y_probs += y_test_probs / n_folds    
    del X_tr, X_val, y_tr, y_val
    model.to('cpu')
    
print(f"\nMean F1 = {f1}")
print(f"\nOut of folds F1 = {f1_score(y_train, y_oof, average='weighted')}")
print(f"\nMean ROC-AUC = {roc_auc}")
print(f"\nOut of folds ROC-AUC = {roc_auc_score(y_train, one_label_to_many(y_oof, n_labels),  multi_class='ovo')}")

test_preds = y_probs.argmax(axis=1)
reverse_repl_dict = {value:key for key, value in repl_dict.items()}
test_data['Категория'] = test_preds
test_data['Категория'] = test_data['Категория'].map(reverse_repl_dict)
n_submission = max([int(file.split('_')[-1].split('.')[0]) for file in os.listdir('./submissions') if 'submission' in file]) + 1
test_data[['id', 'Категория']].to_csv(f'./data/submission_{n_submission}.csv', index=False)

/raid/nanosemantics/tumanov/venv2/lib/python3.8/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=7.
  warnings.warn(


Fold [1/7]


  0%|          | 0/72 [00:00<?, ?it/s]

Epoch [1/10] ... Step [20] ... LR: 0.00000350 ... Grad: 19.07 ... Mean loss: 2.6000

Epoch [1/10] ... Step [40] ... LR: 0.00000683 ... Grad: 19.95 ... Mean loss: 2.3553

Epoch [1/10] ... Step [60] ... LR: 0.00000998 ... Grad: 10.90 ... Mean loss: 2.1689


Epoch [1/10] ... Train ROC-AUC: 0.50 ... Train f1: 0.32 ... Mean train loss: 2.1033


Val ROC-AUC 0.51 ... Val f1: 0.40 ... Mean val loss: 1.5132

-------------------------------------------------------------------------------------------------------------------------



  0%|          | 0/72 [00:00<?, ?it/s]

...

Epoch [5/10] ... Step [20] ... LR: 0.00000623 ... Grad: 66.16 ... Mean loss: 0.3180

Epoch [5/10] ... Step [40] ... LR: 0.00000592 ... Grad: 23.54 ... Mean loss: 0.3032

Epoch [5/10] ... Step [60] ... LR: 0.00000562 ... Grad: 33.39 ... Mean loss: 0.2900


Epoch [5/10] ... Train ROC-AUC: 0.81 ... Train f1: 0.90 ... Mean train loss: 0.2964


Val ROC-AUC 0.76 ... Val f1: 0.84 ... Mean val loss: 0.5702

-------------------------------------------------------------------------------------------------------------------------



  0%|          | 0/72 [00:00<?, ?it/s]

Epoch [6/10] ... Step [20] ... LR: 0.00000514 ... Grad: 48.52 ... Mean loss: 0.2973

Epoch [6/10] ... Step [40] ... LR: 0.00000483 ... Grad: 31.57 ... Mean loss: 0.2819

Epoch [6/10] ... Step [60] ... LR: 0.00000453 ... Grad: 60.02 ... Mean loss: 0.2564


Epoch [6/10] ... Train ROC-AUC: 0.87 ... Train f1: 0.91 ... Mean train loss: 0.2618


Val ROC-AUC 0.77 ... Val f1: 0.85 ... Mean val loss: 0.5554

-------------------------------------------------------------------------------------------------------------------------



  0%|          | 0/72 [00:00<?, ?it/s]

Epoch [7/10] ... Step [20] ... LR: 0.00000405 ... Grad: 110.47 ... Mean loss: 0.2399

Epoch [7/10] ... Step [40] ... LR: 0.00000374 ... Grad: 6.68 ... Mean loss: 0.2057

Epoch [7/10] ... Step [60] ... LR: 0.00000344 ... Grad: 35.78 ... Mean loss: 0.1935


Epoch [7/10] ... Train ROC-AUC: 0.89 ... Train f1: 0.93 ... Mean train loss: 0.1990


Val ROC-AUC 0.76 ... Val f1: 0.85 ... Mean val loss: 0.6008

-------------------------------------------------------------------------------------------------------------------------


Val ROC-AUC 0.77 ... Val f1: 0.85 ... Mean val loss: 0.5554

-------------------------------------------------------------------------------------------------------------------------


Mean F1 = 0.8205962473048001

Out of folds F1 = 0.8216585720681425

Mean ROC-AUC = 0.7770276084523364

Out of folds ROC-AUC = 0.7519888478624376
